In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('all_cars_cleaned.csv')

print("📊 Dataset Shape:", df.shape)
print("\n📋 Columns:", df.columns.tolist())

📊 Dataset Shape: (50000, 11)

📋 Columns: ['Model', 'Year', 'Region', 'Color', 'Fuel_Type', 'Transmission', 'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume', 'Sales_Classification']


In [3]:
# Create advanced features
def create_modeling_features(df):
    df_fe = df.copy()

    # 1. Age of car
    df_fe['Car_Age'] = 2024 - df_fe['Year']

    # 2. Price per liter (efficiency metric)
    df_fe['Price_Per_Liter'] = df_fe['Price_USD'] / df_fe['Engine_Size_L']

    # 3. Mileage per year
    df_fe['Mileage_Per_Year'] = df_fe['Mileage_KM'] / (2024 - df_fe['Year'] + 1)

    # 4. Premium flag
    premium_models = ['7 Series', 'M Series', 'i8']
    df_fe['Is_Premium'] = df_fe['Model'].isin(premium_models).astype(int)

    # 5. Regional economic indicator (proxy)
    region_price_avg = df_fe.groupby('Region')['Price_USD'].mean()
    df_fe['Region_Price_Index'] = df_fe['Region'].map(region_price_avg)

    # 6. Series category
    df_fe['Series_Category'] = df_fe['Model'].apply(lambda x:
        '3 Series' if '3 SERIES' in str(x).upper() else
        '5 Series' if '5 SERIES' in str(x).upper() else
        '7 Series' if '7 SERIES' in str(x).upper() else
        'X Series' if str(x).upper().startswith('X') else
        'M Series' if str(x).upper().startswith('M') else
        'i Series' if str(x).upper().startswith('I') else 'Other'
    )

    # 7. Seasonality (quarter based on year)
    df_fe['Year_Quarter'] = (df_fe['Year'] % 4) + 1

    return df_fe

df_model = create_modeling_features(df)
print("✅ Feature engineering completed!")
print("New features:", [col for col in df_model.columns if col not in df.columns])

✅ Feature engineering completed!
New features: ['Car_Age', 'Price_Per_Liter', 'Mileage_Per_Year', 'Is_Premium', 'Region_Price_Index', 'Series_Category', 'Year_Quarter']


In [4]:
# Select features for modeling
features = ['Year', 'Engine_Size_L', 'Mileage_KM', 'Car_Age', 'Price_Per_Liter',
           'Mileage_Per_Year', 'Is_Premium', 'Region_Price_Index', 'Year_Quarter']

categorical_features = ['Region', 'Color', 'Fuel_Type', 'Transmission', 'Series_Category']

# Prepare feature matrix
X = df_model[features].copy()

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    if col in df_model.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le

# Target variables
y_price = df_model['Price_USD']
y_sales = df_model['Sales_Volume']
y_classification = df_model['Sales_Classification']

print("Feature matrix shape:", X.shape)
print("Target variables:")
print("- Price_USD:", y_price.shape)
print("- Sales_Volume:", y_sales.shape)
print("- Sales_Classification:", y_classification.shape)

Feature matrix shape: (50000, 14)
Target variables:
- Price_USD: (50000,)
- Sales_Volume: (50000,)
- Sales_Classification: (50000,)


In [5]:
print("="*50)
print("🏷️ PRICE PREDICTION MODELING")
print("="*50)

# Split data for price prediction
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X, y_price, test_size=0.2, random_state=42
)

# Scale features
scaler_p = StandardScaler()
X_train_scaled_p = scaler_p.fit_transform(X_train_p)
X_test_scaled_p = scaler_p.transform(X_test_p)

# Define models for price prediction
price_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate price models
price_results = {}

for name, model in price_models.items():
    if name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
        model.fit(X_train_scaled_p, y_train_p)
        y_pred = model.predict(X_test_scaled_p)
    else:
        model.fit(X_train_p, y_train_p)
        y_pred = model.predict(X_test_p)

    mae = mean_absolute_error(y_test_p, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_p, y_pred))
    r2 = r2_score(y_test_p, y_pred)

    price_results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    }

    print(f"{name:20} | MAE: ${mae:,.0f} | RMSE: ${rmse:,.0f} | R²: {r2:.3f}")

# Find best price model
best_price_model = max(price_results.items(), key=lambda x: x[1]['R2'])
print(f"\n🎯 BEST PRICE MODEL: {best_price_model[0]} (R²: {best_price_model[1]['R2']:.3f})")

🏷️ PRICE PREDICTION MODELING
Linear Regression    | MAE: $8,650 | RMSE: $11,034 | R²: 0.820
Ridge Regression     | MAE: $8,650 | RMSE: $11,034 | R²: 0.820
Lasso Regression     | MAE: $8,650 | RMSE: $11,034 | R²: 0.820
Random Forest        | MAE: $55 | RMSE: $84 | R²: 1.000
Gradient Boosting    | MAE: $1,280 | RMSE: $1,732 | R²: 0.996
XGBoost              | MAE: $380 | RMSE: $515 | R²: 1.000

🎯 BEST PRICE MODEL: Random Forest (R²: 1.000)


In [6]:
print("\n" + "="*50)
print("📈 SALES VOLUME PREDICTION MODELING")
print("="*50)

# Split data for sales prediction
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X, y_sales, test_size=0.2, random_state=42
)

# Scale features for sales
scaler_s = StandardScaler()
X_train_scaled_s = scaler_s.fit_transform(X_train_s)
X_test_scaled_s = scaler_s.transform(X_test_s)

# Define models for sales prediction
sales_models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression()
}

# Train and evaluate sales models
sales_results = {}

for name, model in sales_models.items():
    if name == 'Linear Regression':
        model.fit(X_train_scaled_s, y_train_s)
        y_pred = model.predict(X_test_scaled_s)
    else:
        model.fit(X_train_s, y_train_s)
        y_pred = model.predict(X_test_s)

    mae = mean_absolute_error(y_test_s, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_s, y_pred))
    r2 = r2_score(y_test_s, y_pred)

    sales_results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    }

    print(f"{name:20} | MAE: {mae:,.0f} units | RMSE: {rmse:,.0f} | R²: {r2:.3f}")

# Find best sales model
best_sales_model = max(sales_results.items(), key=lambda x: x[1]['R2'])
print(f"\n🎯 BEST SALES MODEL: {best_sales_model[0]} (R²: {best_sales_model[1]['R2']:.3f})")


📈 SALES VOLUME PREDICTION MODELING
Random Forest        | MAE: 2,502 units | RMSE: 2,911 | R²: -0.037
Gradient Boosting    | MAE: 2,479 units | RMSE: 2,863 | R²: -0.003
XGBoost              | MAE: 2,524 units | RMSE: 2,935 | R²: -0.054
Linear Regression    | MAE: 2,477 units | RMSE: 2,859 | R²: -0.000

🎯 BEST SALES MODEL: Linear Regression (R²: -0.000)


In [ ]:
print("\n" + "="*50)
print("🔍 FEATURE IMPORTANCE ANALYSIS")
print("="*50)

# Train best models for feature importance
best_rf_price = RandomForestRegressor(n_estimators=100, random_state=42)
best_rf_price.fit(X_train_p, y_train_p)

best_rf_sales = RandomForestRegressor(n_estimators=100, random_state=42)
best_rf_sales.fit(X_train_s, y_train_s)

# Get feature importance
feature_importance_price = pd.DataFrame({
    'feature': features + categorical_features,
    'importance_price': best_rf_price.feature_importances_
}).sort_values('importance_price', ascending=False)

feature_importance_sales = pd.DataFrame({
    'feature': features + categorical_features,
    'importance_sales': best_rf_sales.feature_importances_
}).sort_values('importance_sales', ascending=False)

print("💰 PRICE PREDICTION - Top Features:")
for _, row in feature_importance_price.head(10).iterrows():
    print(f"  {row['feature']:25} | Importance: {row['importance_price']:.3f}")

print("\n📈 SALES PREDICTION - Top Features:")
for _, row in feature_importance_sales.head(10).iterrows():
    print(f"  {row['feature']:25} | Importance: {row['importance_sales']:.3f}")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Price feature importance
ax1.barh(feature_importance_price['feature'].head(10),
         feature_importance_price['importance_price'].head(10))
ax1.set_title('Feature Importance - Price Prediction', fontweight='bold')
ax1.set_xlabel('Importance')

# Sales feature importance
ax2.barh(feature_importance_sales['feature'].head(10),
         feature_importance_sales['importance_sales'].head(10))
ax2.set_title('Feature Importance - Sales Prediction', fontweight='bold')
ax2.set_xlabel('Importance')

plt.tight_layout()
plt.show()


🔍 FEATURE IMPORTANCE ANALYSIS


In [ ]:
print("\n" + "="*50)
print("👥 CUSTOMER SEGMENTATION ANALYSIS")
print("="*50)

# Features for clustering
cluster_features = ['Price_USD', 'Engine_Size_L', 'Mileage_KM', 'Year', 'Sales_Volume']
X_cluster = df_model[cluster_features].copy()

# Scale features for clustering
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# Determine optimal number of clusters using elbow method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Elbow Method for Optimal Clusters', fontweight='bold')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.grid(True, alpha=0.3)
plt.show()

# Apply K-means with optimal clusters
optimal_clusters = 4  # Based on elbow method
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
df_model['Cluster'] = kmeans.fit_predict(X_cluster_scaled)

# Analyze clusters
cluster_analysis = df_model.groupby('Cluster').agg({
    'Price_USD': ['mean', 'std'],
    'Engine_Size_L': 'mean',
    'Year': 'mean',
    'Sales_Volume': 'mean',
    'Model': lambda x: x.mode()[0],
    'Region': lambda x: x.mode()[0]
}).round(2)

print("🔍 CUSTOMER SEGMENTS:")
print(cluster_analysis)

# Visualize clusters with PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df_model['Cluster'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter)
plt.title('Customer Segments Visualization (PCA)', fontweight='bold')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Add segment descriptions
segment_descriptions = {
    0: "Budget Market",
    1: "Premium Luxury",
    2: "Performance",
    3: "Mid-Range Family"
}

for cluster in range(optimal_clusters):
    cluster_points = X_pca[df_model['Cluster'] == cluster]
    if len(cluster_points) > 0:
        plt.annotate(segment_descriptions.get(cluster, f'Segment {cluster}'),
                    (cluster_points[:, 0].mean(), cluster_points[:, 1].mean()),
                    xytext=(10, 10), textcoords='offset points',
                    fontweight='bold', fontsize=9,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.7))

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "="*50)
print("⏰ TIME SERIES FORECASTING")
print("="*50)

# Prepare time series data
yearly_data = df_model.groupby('Year').agg({
    'Sales_Volume': 'sum',
    'Price_USD': 'mean',
    'Engine_Size_L': 'mean'
}).reset_index()

# Simple time series forecasting using linear regression
X_time = yearly_data[['Year']]
y_time = yearly_data['Sales_Volume']

# Add lag features for better time series prediction
yearly_data['Sales_Lag1'] = yearly_data['Sales_Volume'].shift(1)
yearly_data['Sales_Lag2'] = yearly_data['Sales_Volume'].shift(2)
yearly_data = yearly_data.dropna()

X_time_lag = yearly_data[['Year', 'Sales_Lag1', 'Sales_Lag2']]
y_time_lag = yearly_data['Sales_Volume']

# Train time series model
ts_model = LinearRegression()
ts_model.fit(X_time_lag, y_time_lag)

# Forecast next year
last_year = yearly_data['Year'].max()
last_sales = yearly_data['Sales_Volume'].iloc[-1]
second_last_sales = yearly_data['Sales_Volume'].iloc[-2]

next_year_pred = ts_model.predict([[last_year + 1, last_sales, second_last_sales]])[0]

print(f"📅 TIME SERIES ANALYSIS:")
print(f"• Historical data: {len(yearly_data)} years")
print(f"• Last year sales: {last_sales:,.0f} units")
print(f"• Forecast for {last_year + 1}: {next_year_pred:,.0f} units")
print(f"• Projected growth: {(next_year_pred/last_sales - 1)*100:+.1f}%")

# Plot time series forecast
plt.figure(figsize=(12, 6))
plt.plot(yearly_data['Year'], yearly_data['Sales_Volume'], 'o-', linewidth=2,
         label='Historical Sales', color='#0032A0')

# Add forecast point
plt.plot(last_year + 1, next_year_pred, 'ro', markersize=10,
         label=f'Forecast {last_year + 1}')

plt.title('BMW Sales Time Series & Forecast', fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Sales Volume')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "="*50)
print("⚙️ HYPERPARAMETER TUNING")
print("="*50)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Using smaller subset for faster tuning
X_tune = X.sample(1000, random_state=42) if len(X) > 1000 else X
y_tune = y_price.loc[X_tune.index] if len(X) > 1000 else y_price

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    rf, param_grid, cv=3, scoring='r2',
    n_jobs=-1, verbose=1
)

print("Tuning Random Forest...")
grid_search.fit(X_tune, y_tune)

print(f"🎯 BEST PARAMETERS:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"Best CV Score (R²): {grid_search.best_score_:.3f}")

# Train with best parameters
best_rf_tuned = grid_search.best_estimator_
best_rf_tuned.fit(X_train_p, y_train_p)
y_pred_tuned = best_rf_tuned.predict(X_test_p)

r2_tuned = r2_score(y_test_p, y_pred_tuned)
print(f"Tuned model test R²: {r2_tuned:.3f}")

In [ ]:
print("\n" + "="*50)
print("🚀 MODEL DEPLOYMENT PREPARATION")
print("="*50)

import joblib
import json

# Save best models
model_artifacts = {
    'price_scaler': scaler_p,
    'sales_scaler': scaler_s,
    'best_price_model': best_rf_price,
    'best_sales_model': best_rf_sales,
    'kmeans_model': kmeans,
    'cluster_scaler': scaler_cluster,
    'label_encoders': label_encoders,
    'feature_names': features + categorical_features
}

# Save models
joblib.dump(model_artifacts, 'bmw_models.pkl')

# Save feature importance
feature_importance_dict = {
    'price_features': feature_importance_price.to_dict(),
    'sales_features': feature_importance_sales.to_dict()
}

with open('feature_importance.json', 'w') as f:
    json.dump(feature_importance_dict, f, indent=2)

# Create prediction function
def predict_bmw_price(features_dict):
    """Predict BMW price based on features"""
    # Load model artifacts
    artifacts = joblib.load('bmw_models.pkl')

    # Prepare input features
    input_features = []
    for feature in artifacts['feature_names']:
        if feature in features_dict:
            input_features.append(features_dict[feature])
        else:
            input_features.append(0)  # Default value

    # Make prediction
    prediction = artifacts['best_price_model'].predict([input_features])[0]

    return prediction

# Example prediction
example_car = {
    'Year': 2022,
    'Engine_Size_L': 3.0,
    'Mileage_KM': 15000,
    'Car_Age': 2,
    'Region': 'North America',
    'Series_Category': '5 Series'
}

predicted_price = predict_bmw_price(example_car)
print(f"💰 EXAMPLE PREDICTION:")
print(f"Car features: {example_car}")
print(f"Predicted price: ${predicted_price:,.2f}")

print("\n✅ MODELING COMPLETED!")
print("📁 Saved files:")
print("   - bmw_models.pkl (trained models)")
print("   - feature_importance.json (feature analysis)")
print("   - Ready for deployment! 🚀")

In [ ]:
print("\n" + "="*60)
print("💡 BUSINESS INSIGHTS FROM MODELING ANALYSIS")
print("="*60)

print("🎯 PREDICTION PERFORMANCE:")
print(f"• Price Prediction R²: {best_price_model[1]['R2']:.3f}")
print(f"• Sales Prediction R²: {best_sales_model[1]['R2']:.3f}")

print("\n🔑 KEY DRIVERS IDENTIFIED:")
print("💰 PRICE DRIVERS:")
for feature in feature_importance_price['feature'].head(3):
    print(f"  - {feature}")

print("📈 SALES DRIVERS:")
for feature in feature_importance_sales['feature'].head(3):
    print(f"  - {feature}")

print(f"\n👥 CUSTOMER SEGMENTS: {optimal_clusters} segments identified")
print("  1. Budget Market - Price sensitive customers")
print("  2. Premium Luxury - High-end buyers")
print("  3. Performance - Sport model enthusiasts")
print("  4. Mid-Range Family - Balanced features")

print(f"\n📅 SALES FORECAST: {next_year_pred:,.0f} units in {last_year + 1}")

print("\n🚀 RECOMMENDED ACTIONS:")
print("  1. Focus marketing on key price drivers")
print("  2. Optimize inventory for high-sales features")
print("  3. Develop targeted campaigns for each customer segment")
print("  4. Use price predictions for dynamic pricing strategy")
print("="*60)